In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Storm_Figs_V6.8.2/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_hPa_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_hPa_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time, Birth_Time, Begin_Time, Compl_Time, Death_Time, Peak_Time = [], [], [], [], [], []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Birth_Time.append(Datetime(ET_DF["Genesis Time"][j]))
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Death_Time.append(Datetime(ET_DF["Dissipate Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["Genesis Time"] = Birth_Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Dissipate Time"] = Death_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    return (New_Time)

In [6]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [7]:
# Create Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)

In [8]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [9]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [10]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/composites/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [11]:
# Open Storm Composite Files
Control_A_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
Control_B_Compo_nc = Composite_File('composite_h3_CORI.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
Control_C_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')

In [12]:
# Open Storm Composite Files
RCP45_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP45_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
RCP45_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')

In [13]:
# Open Storm Composite Files
RCP85_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP85_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
RCP85_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004.nc')

In [14]:
Control_A_Compo_nc

<xarray.Dataset>
Dimensions:        (x: 80, y: 80, snapshot: 10159)
Coordinates:
  * x              (x) float64 -11.85 -11.55 -11.25 -10.95 ... 11.25 11.55 11.85
  * y              (y) float64 -11.85 -11.55 -11.25 -10.95 ... 11.25 11.55 11.85
Dimensions without coordinates: snapshot
Data variables: (12/62)
    snap_pathid    (snapshot) int32 ...
    snap_lon       (snapshot) float64 ...
    snap_lat       (snapshot) float64 ...
    snap_time      (snapshot) datetime64[ns] ...
    snap_U850      (snapshot, y, x) float32 ...
    snap_U500      (snapshot, y, x) float32 ...
    ...             ...
    PRECL          (y, x) float32 ...
    FLUT           (y, x) float32 ...
    CLDTOT         (y, x) float32 ...
    TMQ            (y, x) float32 ...
    OMEGA850       (y, x) float32 ...
    OMEGA500       (y, x) float32 ...

In [15]:
# Create DataFrame With Lat Lon Time Data of the Composite Files
def Composite_DF(Compo_nc, ABC):
    Snap_Time = pandas.Series(Compo_nc.snap_time)
    Snap_Lon = pandas.Series(Compo_nc.snap_lon)
    Snap_Lat = pandas.Series(Compo_nc.snap_lat)
    Snap_PathID = pandas.Series(Compo_nc.snap_pathid)
    Index = numpy.arange(0,len(Snap_Time),1)
    ABC_List = []
    for m in range(len(Index)):
        ABC_List.append(ABC)
    Compo_DF = pandas.DataFrame({"Orig Index": Index, "ABC": ABC_List, \
    "Time": Snap_Time, "Lon": Snap_Lon, "Lat": Snap_Lat, "PathID": Snap_PathID})
    return (Compo_DF)

In [16]:
# Combine Composite DFs
def Combine_Compo_DF(Compo_A, Compo_B, Compo_C):
    Compo_DF_A = Composite_DF(Compo_A, "A")
    Compo_DF_B = Composite_DF(Compo_B, "B")
    Compo_DF_C = Composite_DF(Compo_C, "C")
    Compo_DF = pandas.concat([Compo_DF_A, Compo_DF_B, Compo_DF_C]).reset_index()
    Compo_DF = Compo_DF.drop("index", axis=1)
    return (Compo_DF)

In [17]:
Control_Compo = Combine_Compo_DF(Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [18]:
RCP45_Compo = Combine_Compo_DF(RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [19]:
RCP85_Compo = Combine_Compo_DF(RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [20]:
RCP45_Compo[RCP45_Compo["ABC"] == "A"]

,Orig Index,ABC,Time,Lon,Lat,PathID
0,0,A,2070-01-29 12:00:00,-55.37,40.08,0
1,1,A,2070-01-29 18:00:00,-52.23,39.59,0
2,2,A,2070-01-30 00:00:00,-49.08,39.11,0
3,3,A,2070-01-30 06:00:00,-47.40,38.08,0
4,4,A,2070-01-30 12:00:00,-45.73,37.48,0
...,...,...,...,...,...,...
6962,6962,A,2100-12-25 18:00:00,-22.75,26.75,227
6963,6963,A,2100-12-26 00:00:00,-21.75,27.25,227
6964,6964,A,2100-12-26 06:00:00,-20.50,28.25,227
6965,6965,A,2100-12-26 12:00:00,-19.50,29.50,227


In [21]:
# Change Year of Data
def Reverse_Update_Year(New_Time, Year_Diff):
    Year_Orig = New_Time.year + Year_Diff
#    print (Year_Orig)
    Orig_Time = New_Time.replace(year=Year_Orig)
    return (Orig_Time)

In [22]:
# Create Function to Find Year Diff
def Year_Diff_Find(New_Time):
    Years = [1900,1930,1960,2000,2031,2062,2100,2131,2162,2193]
    New_Time_Index = -728
    for i in range(len(Years)):
        if i < 3:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 1985 - Years[i]
                New_Time_Index = i
        elif i < 6:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
        else:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
    if New_Time_Index % 3 == 0:
        ABC = "A"
    elif New_Time_Index % 3 == 1:
        ABC = "B"
    elif New_Time_Index % 3 == 2:
        ABC = "C"
    return (int(Year_Diff), ABC)

In [23]:
# Create Function to Find Indexes of Composite Data For Selected Storm
def Find_Composite_Data(Code, Data_DF, Compo_DF):
    DF_Storm = Find_Storm(Data_DF, Code)
    New_Time = DF_Storm["Time(Z)"]
    Lat = DF_Storm["Lat"]
    Lon = DF_Storm["Lon"]
    SLP = DF_Storm["SLP(hPa)"]
    Windspeed = DF_Storm["Winds(m/s)"]
    Storm_Phase = DF_Storm["Storm Phase"]
    Compo_Indexes = numpy.zeros(len(New_Time))
    for i in range(len(New_Time)):
        Year_Diff, ABC = Year_Diff_Find(New_Time[0])
        Orig_Time = Reverse_Update_Year(New_Time[i], Year_Diff)
# Find Possible Storms that Occur at the Same Time
        Compo_Storm = Compo_DF[(Compo_DF["ABC"] == ABC) & (Compo_DF["Time"] == Orig_Time)].reset_index()
# If No Storm Found:
        if len(Compo_Storm) == 0:
            Compo_Indexes[i] = -728
# Storms Found:
        else:
            Dist_Min = [7428,-728]
            for c in range(len(Compo_Storm)):
                Dist = Find_Distance(Lat[i], Compo_Storm["Lat"][c], Lon[i], Compo_Storm["Lon"][c])
# Find Storm Closest to Storm Center
                if Dist < Dist_Min[0]:
# At Most 300km of Error in Location Permitted
                    if Dist < 300:
                        Dist_Min = [Dist, Compo_Storm["Orig Index"][c]]
                    else:
                        Dist_Min = [Dist, -728]
            Compo_Indexes[i] = Dist_Min[1]
    DF_Storm_Compo_Init = pandas.DataFrame({"Compo Index": Compo_Indexes, "Time": New_Time, \
    "Lon": Lon, "Lat": Lat, "SLP(hPa)": SLP, "Sfc Winds(m/s)": Windspeed, "Storm Phase": Storm_Phase})
# Remove Datapoints With Missing Compo Index
    DF_Storm_Compo = DF_Storm_Compo_Init[DF_Storm_Compo_Init["Compo Index"] >= 0].reset_index()
    DF_Storm_Compo = DF_Storm_Compo.drop("index", axis=1)
    return (DF_Storm_Compo)

In [24]:
Control_A_Compo_nc.snap_U850[3987][38]

<xarray.DataArray 'snap_U850' (x: 80)>
[80 values with dtype=float32]
Coordinates:
  * x        (x) float64 -11.85 -11.55 -11.25 -10.95 ... 10.95 11.25 11.55 11.85
    y        float64 -0.45

In [25]:
def Windspeed_850hPa(Compo_nc, Compo_Index):
    U850 = numpy.array(Compo_nc.snap_U850[int(Compo_Index)])
    V850 = numpy.array(Compo_nc.snap_V850[int(Compo_Index)])
    Snap_850 = numpy.sqrt(U850 **2 + V850 **2)
    return (Snap_850)

In [26]:
# Find 850hPa Max Windspeed and Wind Field Size at Each 6 Hourly Data Point
def Wind_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
    SLP = DF_Storm_Compo["SLP(hPa)"]
    Lat = DF_Storm_Compo["Lat"]
    Lon = DF_Storm_Compo["Lon"]
    x_Snap = Compo_nc.x
    y_Snap = Compo_nc.y
#
# Create Array to Store Data
    Wind_Field_Info = numpy.zeros((2,len(Compo_Index)))
    Wind_Field_Info[0] = SLP
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Windspeed Snap From Compo_nc
        Snap_850 = Windspeed_850hPa(Compo_nc, Compo_Index[k])
# Find Maximum 850hPa Windspeed
        Windspeed_850 = numpy.max(Snap_850)
        Wind_Field_Info[1][k] = Windspeed_850
#
# Add Wind Field Info Into DF Storm Compo
    DF_Storm_Compo["850hPa Winds"] = Wind_Field_Info[1]
    return (DF_Storm_Compo)

In [27]:
# Create Function to Find Compo Info
def Find_Compo_Info(Code, Data_DF, ET_DF, Compo_DF, Compo_nc):
    DF_Storm_Compo = Find_Composite_Data(Code, Data_DF, Compo_DF)
    DF_Storm_Compo = Wind_Field_Find(DF_Storm_Compo, Compo_nc)
#
# Find Compo Index of Storm Peak
    DF_Storm_Compo_TC = DF_Storm_Compo[DF_Storm_Compo["Storm Phase"] == "Tropical"]
    Max_Wind_850 = numpy.max(numpy.array(DF_Storm_Compo_TC["850hPa Winds"]))
    Storm_Peak_Index = DF_Storm_Compo_TC[DF_Storm_Compo_TC["850hPa Winds"] >= Max_Wind_850].reset_index()["Compo Index"][0]
# Find Compo Index of ET Begin
    DF_Storm_Compo_Trans = DF_Storm_Compo[DF_Storm_Compo["Storm Phase"] == "Transition"].reset_index()
    Trans_Length = len(DF_Storm_Compo_Trans)
    if Trans_Length > 0:
        ET_Begin_Index = DF_Storm_Compo_Trans["Compo Index"][0]
# Find Compo Index of ET Midpoint
        Mid = int(Trans_Length / 2)
        ET_Mid_Index = DF_Storm_Compo_Trans["Compo Index"][Mid]
    else:
        DF_Storm_Compo_ExTC = DF_Storm_Compo[DF_Storm_Compo["Storm Phase"] == "Extratropical"].reset_index()
        ET_Begin_Index = DF_Storm_Compo_ExTC["Compo Index"][0]
        ET_Mid_Index = DF_Storm_Compo_ExTC["Compo Index"][0]
# Find Compo Index of ET Complete
    DF_Storm_Compo_ExTC = DF_Storm_Compo[DF_Storm_Compo["Storm Phase"] == "Extratropical"].reset_index()
    ET_Compl_Index = DF_Storm_Compo_ExTC["Compo Index"][0]
    Time_Indexes = [Storm_Peak_Index, ET_Begin_Index, ET_Mid_Index, ET_Compl_Index]
    return (DF_Storm_Compo, Time_Indexes)

In [28]:
# Create Function to Find Windspeed Map
def Wind_Map_Find(Windspeed_Snap):
# Initial Indexes
    Wind_Map = numpy.zeros((80,80))
    i_Init = 0
    j_Init = 0
    for i in range(len(Wind_Map)):
        for j in range(len(Wind_Map[0])):
# Find Index inside the Windspeed Snap Array
            xi = int(i+i_Init)
            yj = int(j+j_Init)
            Windspeed = Windspeed_Snap[yj][xi]
            Wind_Map[i][j] = Windspeed
    return (Wind_Map)

In [29]:
# Create Function to Combine Windspeed Maps
def Wind_Map_Combine(Subset_Codes, Data_DF, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Wind_Map_Sum = numpy.zeros((len(Subset_Codes),4,80,80))
    for n in range(len(Subset_Codes)):
        Code = Subset_Codes[n]
        ABC = ET_DF[ET_DF["Code"] == Code].reset_index()["ABC"][0]
        if ABC == "A":
            Compo_nc = Compo_nc_A
        elif ABC == "B":
            Compo_nc = Compo_nc_B
        elif ABC == "C":
            Compo_nc = Compo_nc_C
        DF_Storm_Compo, Time_Indexes = Find_Compo_Info(Code, Data_DF, ET_DF, Compo_DF, Compo_nc)
        for t in range(len(Time_Indexes)):
            Windspeed_Snap = numpy.array(Windspeed_850hPa(Compo_nc, Time_Indexes[t]))
            Wind_Map = Wind_Map_Find(Windspeed_Snap)
            Wind_Map_Sum[n][t] = Wind_Map
    Wind_Map_Mean = numpy.mean(Wind_Map_Sum, axis=0)
    return (Wind_Map_Mean)

In [30]:
# Define Colours and Bounds
Windspeed_Colours = ['deepskyblue', 'cyan', 'lightseagreen', 'limegreen', 'gold', 'darkorange', 'red', 'darkviolet']
Windspeed_Bounds = numpy.array([13,18,25,33,41,50,60])

In [31]:
# Create Function to Plot Windspeed Snapshot Maps
def Windspeed_Map(Wind_Map, x_Snap, y_Snap, Time_Label, x_Label):
# Add Corrdinate Lines
    pyplot.plot([-728,728], [0,0], c='black', linestyle='-', linewidth=0.8)
    pyplot.plot([0,0], [-728,728], c='black', linestyle='-', linewidth=0.8)
# Initial Indexes
    i_Init = 0
    j_Init = 0
    i_Fin = 80
    j_Fin = 80
# Plot Contour Map
    pyplot.contourf(x_Snap, y_Snap, Wind_Map, levels=Windspeed_Bounds, colors=Windspeed_Colours, extend='both')
# Formatting
    x_Min = float(x_Snap[i_Init])
    x_Max = float(x_Snap[i_Fin-1])
    y_Min = float(y_Snap[j_Init])
    y_Max = float(y_Snap[j_Fin-1])
    pyplot.xlim(x_Min,x_Max)
    pyplot.ylim(y_Min,y_Max)
    if x_Label == True:
        pyplot.xlabel('Lon Diff From Storm Center', fontsize=12)
    pyplot.ylabel('Lat Diff From Storm Center', fontsize=12)
#    pyplot.grid(linewidth=0.3, color='silver', linestyle='-')
    Title = Time_Label
    pyplot.title(Title, fontsize=18)

In [32]:
# Plot Four Windspeed Snap Plots
def Windspeed_Snap_Plots(Wind_Maps, Compo_nc, Time_Labels, Model, Savefig):
# x and y Values
    x_Snap = Compo_nc.x
    y_Snap = Compo_nc.y
#

# Create Fig
    Fig = pyplot.figure(figsize=(16,16))
# First Plot
    Axis = Fig.add_subplot(2,2,1)
    Windspeed_Map(Wind_Maps[0], x_Snap, y_Snap, Time_Labels[0], False)
#
# Second Plot
    Axis = Fig.add_subplot(2,2,2)
    Windspeed_Map(Wind_Maps[1], x_Snap, y_Snap, Time_Labels[1], False)
#
# Colour Bar
    C_Map = mcolors.LinearSegmentedColormap.from_list("850hPa Windspeeds", Windspeed_Colours, N=len(Windspeed_Colours))
    Norm = mcolors.BoundaryNorm(Windspeed_Bounds, C_Map.N, extend='both')
    Cax = Axis.inset_axes([-0.85, -0.095, 1.5, 0.025], transform=Axis.transAxes)
    Fig.colorbar(ScalarMappable(cmap=C_Map, norm=Norm), orientation='horizontal', spacing='proportional', \
    cax=Cax, label='850hPa Windspeeds (m/s)', shrink=0.7)
#
# Third Plot
    Axis = Fig.add_subplot(2,2,3)
    Windspeed_Map(Wind_Maps[2], x_Snap, y_Snap, Time_Labels[2], True)
#
# Fourth Plot
    Axis = Fig.add_subplot(2,2,4)
    Windspeed_Map(Wind_Maps[3], x_Snap, y_Snap, Time_Labels[3], True)
#
# Save Fig
    if Savefig == True:
        Figname = str(Model + '_Wind_Map_Composite.png')
        Fig.savefig(Output_Diri+Figname, bbox_inches='tight')

In [33]:
# Create Subsets of Dataframe Based on Selection Criteria

In [34]:
Control_Subset_Codes = \
['TC190003', 'TC190004', 'TC190105', 'TC190205', 'TC190406', \
'TC190503', 'TC191304', 'TC191608', 'TC192209', 'TC192215', \
'TC192305', 'TC192711', 'TC193210', 'TC193811', 'TC194002', \
'TC194005', 'TC194007', 'TC194008', 'TC194010', 'TC194203', \
'TC194703', 'TC195006', 'TC195205', 'TC195609', 'TC196410', \
'TC196509', 'TC197709', 'TC197807', 'TC198413', 'TC198503']

In [35]:
Control_Subset_Codes = ['TC191304']

In [36]:
#Control_Subset = Control_ET[(Control_ET['Peak SLP'] <= 960) & (Control_ET['ET Begin SLP'] <= 990) & (Control_ET['ET Complete SLP'] <= 1000) & (Control_ET['Peak Lat'] <= 40)]
#Control_Subset_Codes = list(Control_Subset['Code'])
Control_Subset = Control_ET[Control_ET['Code'].isin(Control_Subset_Codes)]
Control_Subset

,Code,Name,ABC,Trans Type,Genesis Time,ET Begin Time,ET Complete Time,Dissipate Time,Peak Time,Peak SLP,...,Genesis Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat,Dissipate SLP,Dissipate Lon,Dissipate Lat
40,TC191304,Melody,A,1,1913-08-24 18:00:00,1913-09-05 18:00:00,1913-09-08 18:00:00,1913-09-09 12:00:00,1913-09-01 18:00:00,894.09,...,12.08,932.19,-71.04,31.75,968.71,-51.01,50.86,971.93,-46.75,54.75


In [37]:
RCP45_Subset_Codes = \
['TC200105', 'TC200510', 'TC200705', 'TC200811', 'TC200903', \
'TC201004', 'TC201404', 'TC202006', 'TC202009', 'TC202010', \
'TC202308', 'TC203001', 'TC203301', 'TC203404', 'TC203603', \
'TC204308', 'TC204702', 'TC205002', 'TC205115', 'TC206402', \
'TC206607', 'TC207009', 'TC208209', 'TC208303', 'TC208503', \
'TC208505', 'TC208507', 'TC208705', 'TC208708', 'TC209108']

In [38]:
#RCP45_Subset = RCP45_ET[(RCP45_ET['Peak SLP'] <= 960) & (RCP45_ET['ET Begin SLP'] <= 995) & (RCP45_ET['ET Complete SLP'] <= 1005)]
#RCP45_Subset_Codes = list(RCP45_Subset['Code'])
RCP45_Subset = RCP45_ET[RCP45_ET['Code'].isin(RCP45_Subset_Codes)]
RCP45_Subset

,Code,Name,ABC,Trans Type,Genesis Time,ET Begin Time,ET Complete Time,Dissipate Time,Peak Time,Peak SLP,...,Genesis Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat,Dissipate SLP,Dissipate Lon,Dissipate Lat
3,TC200105,Brandon,A,1,2001-09-17 00:00:00,2001-09-29 00:00:00,2001-09-29 12:00:00,2001-09-30 18:00:00,2001-09-28 12:00:00,928.97,...,9.16,966.21,-87.42,31.07,989.30,-86.53,33.40,1002.57,-85.00,38.25
10,TC200510,Anna,A,1,2005-08-31 12:00:00,2005-09-09 12:00:00,2005-09-11 06:00:00,2005-09-11 06:00:00,2005-09-07 00:00:00,897.70,...,11.75,933.51,-60.82,37.35,959.00,-43.52,47.85,959.00,-43.52,47.85
13,TC200705,Tess,A,1,2007-10-10 06:00:00,2007-10-19 18:00:00,2007-10-22 18:00:00,2007-10-23 00:00:00,2007-10-19 12:00:00,916.14,...,35.12,924.91,-44.37,33.20,992.79,-6.25,42.97,1000.85,-2.39,42.31
19,TC200811,Naomi,A,1,2008-09-24 18:00:00,2008-10-02 12:00:00,2008-10-05 06:00:00,2008-10-05 12:00:00,2008-10-01 06:00:00,914.43,...,14.37,926.67,-51.94,27.77,964.49,-21.75,52.50,967.41,-16.25,56.75
21,TC200903,Emmelyn,A,1,2009-08-20 18:00:00,2009-08-30 00:00:00,2009-08-31 12:00:00,2009-09-01 00:00:00,2009-08-26 18:00:00,938.74,...,10.15,948.16,-61.86,40.75,975.30,-45.84,50.14,979.30,-38.14,52.24
23,TC201004,Nabil,A,1,2010-10-26 12:00:00,2010-11-05 12:00:00,2010-11-06 06:00:00,2010-11-08 06:00:00,2010-11-02 00:00:00,953.30,...,22.06,991.23,-49.05,44.33,1004.45,-39.59,42.77,1014.15,-32.50,33.25
29,TC201404,Oscar,A,1,2014-08-28 00:00:00,2014-09-09 06:00:00,2014-09-10 06:00:00,2014-09-10 06:00:00,2014-09-07 18:00:00,946.70,...,11.60,960.91,-35.49,45.24,983.30,-23.24,50.47,983.30,-23.24,50.47
43,TC202006,Grace,A,1,2020-08-10 12:00:00,2020-08-19 18:00:00,2020-08-20 06:00:00,2020-08-23 00:00:00,2020-08-18 18:00:00,947.90,...,17.00,988.26,-72.59,43.04,998.72,-67.32,47.37,1007.08,-15.25,54.25
45,TC202009,Kinen,A,1,2020-09-17 18:00:00,2020-09-28 00:00:00,2020-09-28 18:00:00,2020-09-29 00:00:00,2020-09-24 00:00:00,924.21,...,12.08,961.75,-65.08,41.88,974.92,-57.37,44.79,978.80,-53.36,46.55
46,TC202010,Lydia,A,1,2020-10-03 06:00:00,2020-10-14 18:00:00,2020-10-15 06:00:00,2020-10-15 12:00:00,2020-10-13 06:00:00,948.10,...,10.57,983.57,-43.09,43.22,995.22,-35.84,48.52,994.39,-30.75,52.00


In [39]:
RCP85_Subset_Codes = \
['TC210105', 'TC210806', 'TC211001', 'TC211305', 'TC211308', \
'TC211405', 'TC212402', 'TC213409', 'TC213606', 'TC213607', \
'TC213903', 'TC214005', 'TC214504', 'TC214807', 'TC215407', \
'TC215907', 'TC215911', 'TC216004', 'TC216105', 'TC216303', \
'TC216706', 'TC216805', 'TC217106', 'TC217204', 'TC218001', \
'TC218206', 'TC219002', 'TC219007', 'TC219102', 'TC219103']

In [40]:
#RCP85_Subset = RCP85_ET[(RCP85_ET['Peak SLP'] <= 960) & (RCP85_ET['ET Begin SLP'] <= 995) & (RCP85_ET['ET Complete SLP'] <= 1005)]
#RCP85_Subset_Codes = list(RCP85_Subset['Code'])
RCP85_Subset = RCP85_ET[RCP85_ET['Code'].isin(RCP85_Subset_Codes)]
RCP85_Subset

,Code,Name,ABC,Trans Type,Genesis Time,ET Begin Time,ET Complete Time,Dissipate Time,Peak Time,Peak SLP,...,Genesis Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat,Dissipate SLP,Dissipate Lon,Dissipate Lat
3,TC210105,Nicole,A,1,2101-08-31 12:00:00,2101-09-12 18:00:00,2101-09-14 18:00:00,2101-09-14 18:00:00,2101-09-08 18:00:00,902.97,...,9.92,934.13,-60.07,35.72,962.56,-47.38,49.02,962.56,-47.38,49.02
15,TC210806,Esther,A,1,2108-11-02 12:00:00,2108-11-11 06:00:00,2108-11-12 00:00:00,2108-11-13 00:00:00,2108-11-10 18:00:00,957.55,...,31.29,966.57,-31.84,44.42,978.85,-22.88,49.50,998.03,1.75,47.25
17,TC211001,Carson,A,1,2110-08-18 00:00:00,2110-08-30 12:00:00,2110-08-31 12:00:00,2110-09-02 00:00:00,2110-08-27 00:00:00,920.65,...,15.43,938.26,-56.41,43.25,977.40,-38.13,47.96,995.80,-7.25,54.00
22,TC211305,Hazel,A,1,2113-06-10 06:00:00,2113-06-19 00:00:00,2113-06-19 18:00:00,2113-06-19 18:00:00,2113-06-18 00:00:00,959.56,...,12.06,968.89,-67.21,38.58,980.91,-60.91,44.87,980.91,-60.91,44.87
24,TC211308,Luca,A,1,2113-09-12 12:00:00,2113-09-21 00:00:00,2113-09-25 06:00:00,2113-09-26 18:00:00,2113-09-18 00:00:00,911.77,...,13.53,956.55,-44.48,31.57,977.74,-48.83,53.69,995.67,-39.25,55.00
27,TC211405,Alberto,A,1,2114-08-13 06:00:00,2114-08-26 00:00:00,2114-08-28 00:00:00,2114-08-29 18:00:00,2114-08-25 00:00:00,915.17,...,14.05,928.16,-60.60,37.45,973.34,-46.87,49.91,988.63,-24.75,54.25
45,TC212402,Harrison,A,1,2124-09-19 12:00:00,2124-10-02 00:00:00,2124-10-07 00:00:00,2124-10-07 00:00:00,2124-09-30 06:00:00,923.72,...,9.33,924.02,-53.05,25.97,988.26,-27.32,49.92,988.26,-27.32,49.92
67,TC213409,Daniel,B,1,2134-09-13 06:00:00,2134-09-20 06:00:00,2134-09-22 00:00:00,2134-09-22 12:00:00,2134-09-18 06:00:00,923.37,...,14.91,956.11,-77.99,34.43,991.81,-70.49,42.75,997.93,-64.05,44.14
74,TC213606,Frederick,B,1,2136-09-23 00:00:00,2136-10-01 12:00:00,2136-10-03 18:00:00,2136-10-04 00:00:00,2136-09-30 00:00:00,942.75,...,14.42,957.06,-48.11,32.42,984.42,-29.69,53.45,992.91,-29.25,57.25
75,TC213607,Gwyneth,B,1,2136-10-03 12:00:00,2136-10-13 06:00:00,2136-10-21 18:00:00,2136-10-24 06:00:00,2136-10-14 00:00:00,920.08,...,14.04,930.46,-77.57,26.36,987.64,-31.04,49.12,1013.97,-22.00,49.75


In [41]:
Control_Subset_Wind_Map = Wind_Map_Combine(Control_Subset_Codes, Control_Data, Control_ET, Control_Compo, Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)
#Control_Subset_Wind_Map

NameError: name 'y' is not defined

In [ ]:
RCP45_Subset_Wind_Map = Wind_Map_Combine(RCP45_Subset_Codes, RCP45_Data, RCP45_ET, RCP45_Compo, RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)
#RCP45_Subset_Wind_Map

In [ ]:
RCP85_Subset_Wind_Map = Wind_Map_Combine(RCP85_Subset_Codes, RCP85_Data, RCP85_ET, RCP85_Compo, RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)
#RCP85_Subset_Wind_Map

In [ ]:
Time_Labels = ["Storm Peak", "ET Begin", "ET Midpoint", "ET Complete"]

In [ ]:
Windspeed_Snap_Plots(Control_Subset_Wind_Map, Control_A_Compo_nc, Time_Labels, "Control", True)

In [ ]:
Windspeed_Snap_Plots(RCP45_Subset_Wind_Map, RCP45_A_Compo_nc, Time_Labels, "RCP4.5", True)

In [ ]:
Windspeed_Snap_Plots(RCP85_Subset_Wind_Map, RCP85_A_Compo_nc, Time_Labels, "RCP8.5", True)